In [ ]:
using Pkg, Revise
Pkg.activate("..")
using ISOKANN, Plots
Plots.default(fmt=:png)

In [ ]:
# we start by loading the .pdb file into a ::Molly.System
sys = PDB_ACEMD()  # Alanine-Dipeptide helper function taking care of the forcefields, boundary ...

In [ ]:
# and plug it into a ::MollyLangevin which specifies a simulation (integrator, timesteps, length)
sim = MollyLangevin(;sys)

In [ ]:
# solving the simulation returns a matrix with the atoms positions in each column
traj = solve(sim);

In [ ]:
# the dihedral angles depicted in the ramachandran plot offer a good visualization of the state of the molecule
scatter_ramachandran(traj, fmt=:png)

In [ ]:
# create the actual ISOKANN simulation
iso = IsoRun(;sim, loggers=[])

In [ ]:
# training runs for about 1.5 minutes
@time run!(iso, showprogress = false)

In [ ]:
plot_learning(iso)

In [ ]:
x0 = ISOKANN.getcoords(sim)
chi = iso.model
path = @time reactionpath(sim, x0, chi, dt=0.0001)
scatter_ramachandran(path)

In [ ]:
ISOKANN.savecoords(sys, path, "reactionpath.pdb")